In [4]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome
from ipynb.fs.defs.imdb_data import *

In [5]:
def get_fw_data(title):
    title = title.replace(" ", "+")
    
    browser = Chrome(fr'{Paths.driver}')
    browser.get(f'https://www.filmweb.pl/search?q={title}')
    movie_dict = {}

    html = browser.find_element_by_tag_name('html')
    html.find_element_by_id('didomi-notice-agree-button').click()

    results_list = html.find_element_by_class_name('resultsList.hits')
    results = results_list.find_elements_by_class_name('hits__item')
    movie_dict['polish_title'] = results[0].find_element_by_class_name('filmPreview__title').text
    try:
        movie_dict['original_title'] = results[0].find_element_by_class_name('filmPreview__originalTitle').text
    except Exception:
        movie_dict['original_title'] = movie_dict['polish_title']
    movie_dict['year'] = results[0].find_element_by_class_name('filmPreview__year').text
    movie_dict['rating'] = results[0].find_element_by_class_name('rateBox__rate').text

    browser.close()
    
    movie_fw_df = pd.DataFrame.from_dict([movie_dict])
    
    return movie_fw_df


In [46]:
column_names = ['polish_title', 'original_title', 'year','rating']
df_fw = pd.DataFrame(columns = column_names)

titles = ['Uncut Gems', 'Dzień Świra', 'Fear and Loathing in Las Vegas']

for title in titles:
    df_fw = pd.concat([df_fw, get_fw_data(title)], ignore_index=True)
    

imdb_data = get_imdb_data(df_fw)
df_imdb = pd.DataFrame.from_dict(imdb_data)

fw_imdb_join = pd.merge(
    left=df_fw,
    right=df_imdb,
    how='left',
    left_on='original_title',
    right_on='title'
)

fw_imdb_join.rename(columns={'rating_x': 'rating_fw', 'rating_y': 'rating_imdb'}, inplace=True)
fw_imdb_join.drop('title', 1, inplace=True)

data_merged = pd.read_csv(
    fr'{Paths.data}\movies_data_updated.csv',
    sep=',',
    decimal='.',
    encoding='utf-8'
)

df_new = pd.concat([data_merged, fw_imdb_join], ignore_index=True)
df_new.head()

<ipython-input-40-418e1c05235d>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(r'\Users\micha\project_filmweb\drivers\chromedriver')
<ipython-input-40-418e1c05235d>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  html = browser.find_element_by_tag_name('html')
D:\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:187: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
D:\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:445: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
D:\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:463: UserWarning: find_elemen

,polish_title,original_title,year,rating_fw,genre,direction,screenplay,rating_imdb
0,Skazani na Shawshank,The Shawshank Redemption,1994,"8,76",Dramat,Frank Darabont,"Stephen King, Frank Darabont",9.3
1,Nietykalni,Intouchables,2011,"8,61",Biograficzny / Dramat / Komedia,"Olivier Nakache, Éric Toledano","Olivier Nakache, Philippe Pozzo di Borgo, Éric...",8.5
2,Zielona mila,The Green Mile,1999,"8,60",Dramat,Frank Darabont,"Stephen King, Frank Darabont",8.6
3,Ojciec chrzestny,The Godfather,1972,"8,59",Dramat / Gangsterski,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola, Mario Puzo",9.2
4,Dwunastu gniewnych ludzi,12 Angry Men,1957,"8,56",Dramat sądowy,Sidney Lumet,"Reginald Rose, Reginald Rose",9.0


In [47]:
df_new

,polish_title,original_title,year,rating_fw,genre,direction,screenplay,rating_imdb
0,Skazani na Shawshank,The Shawshank Redemption,1994,"8,76",Dramat,Frank Darabont,"Stephen King, Frank Darabont",9.3
1,Nietykalni,Intouchables,2011,"8,61",Biograficzny / Dramat / Komedia,"Olivier Nakache, Éric Toledano","Olivier Nakache, Philippe Pozzo di Borgo, Éric...",8.5
2,Zielona mila,The Green Mile,1999,"8,60",Dramat,Frank Darabont,"Stephen King, Frank Darabont",8.6
3,Ojciec chrzestny,The Godfather,1972,"8,59",Dramat / Gangsterski,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola, Mario Puzo",9.2
4,Dwunastu gniewnych ludzi,12 Angry Men,1957,"8,56",Dramat sądowy,Sidney Lumet,"Reginald Rose, Reginald Rose",9.0
...,...,...,...,...,...,...,...,...
1340,Pearl Harbor,Pearl Harbor,2001,"7,41",Melodramat / Wojenny,Michael Bay,Randall Wallace,6.2
1341,Hubal,Hubal,1973,"7,41",Biograficzny / Wojenny,Bohdan Poreba,"Ryszard Filipski, Bohdan Poreba, Jan Józef Szc...",6.9
1342,Nieoszlifowane diamenty,Uncut Gems,2019,"6,8",DramatKomediaKryminał,"Benny Safdie, Josh Safdie","Ronald Bronstein, Josh Safdie, Benny Safdie",7.4
1343,Dzień świra,Dzień świra,2002,"7,8",DramatKomediaPsychologiczny,Marek Koterski,Marek Koterski,8.1
